In [1]:
#Import Dependencies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
import findspark
findspark.init()
from pyspark.sql import SparkSession
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense
from keras import regularizers

/var/folders/_l/b2ndqf2j6h5gcz0gbj9zfl4r0000gn/T/ipykernel_77993/2341801833.py:2: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
#Create Spark Session
spark = SparkSession.builder.appName("Read CSV").getOrCreate()


24/03/11 18:37:54 WARN Utils: Your hostname, Aarons-MacBook-Pro.local resolves to a loopback address: 127.0.0.1; using 192.168.1.5 instead (on interface en0)
24/03/11 18:37:54 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/11 18:37:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
#Read in CSV

file_path = "Credit_Score_Data/credit_risk.csv"

df = spark.read.csv(file_path, header=True, inferSchema=True, sep=",")

df.show()

+----------+-------------+---------------------+-----------------+-----------------+----------+---------+-------------+-----------+-------------------+-------------------------+--------------------------+
|person_age|person_income|person_home_ownership|person_emp_length|      loan_intent|loan_grade|loan_amnt|loan_int_rate|loan_status|loan_percent_income|cb_person_default_on_file|cb_person_cred_hist_length|
+----------+-------------+---------------------+-----------------+-----------------+----------+---------+-------------+-----------+-------------------+-------------------------+--------------------------+
|        22|        59000|                 RENT|            123.0|         PERSONAL|         D|    35000|        16.02|          1|               0.59|                        Y|                         3|
|        21|         9600|                  OWN|              5.0|        EDUCATION|         B|     1000|        11.14|          0|                0.1|                        N|   

In [4]:
#Convert Spark DF to Pandas
credit_df= df.toPandas()

credit_df.head()

,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_status,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,22,59000,RENT,123.0,PERSONAL,D,35000,16.02,1,0.59,Y,3
1,21,9600,OWN,5.0,EDUCATION,B,1000,11.14,0,0.10,N,2
2,25,9600,MORTGAGE,1.0,MEDICAL,C,5500,12.87,1,0.57,N,3
3,23,65500,RENT,4.0,MEDICAL,C,35000,15.23,1,0.53,N,2
4,24,54400,RENT,8.0,MEDICAL,C,35000,14.27,1,0.55,Y,4


In [5]:
#End Spark Session
spark.stop()

In [6]:
credit_df.dropna(inplace=True)

In [7]:
credit_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 28638 entries, 0 to 32580
Data columns (total 12 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   person_age                  28638 non-null  int32  
 1   person_income               28638 non-null  int32  
 2   person_home_ownership       28638 non-null  object 
 3   person_emp_length           28638 non-null  float64
 4   loan_intent                 28638 non-null  object 
 5   loan_grade                  28638 non-null  object 
 6   loan_amnt                   28638 non-null  int32  
 7   loan_int_rate               28638 non-null  float64
 8   loan_status                 28638 non-null  int32  
 9   loan_percent_income         28638 non-null  float64
 10  cb_person_default_on_file   28638 non-null  object 
 11  cb_person_cred_hist_length  28638 non-null  int32  
dtypes: float64(3), int32(5), object(4)
memory usage: 2.3+ MB


In [8]:

# Convert categorical data to numeric
df_credit= pd.get_dummies(credit_df)

In [9]:
df_prep = df_credit.drop("loan_status", axis = 1)

In [10]:
# Split data into features and target arrays
X = df_prep
y = credit_df["loan_status"]


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

In [12]:
# Create a StandardScaler instance
scaler = StandardScaler()

#Fit and scale the data
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [13]:

#  Define the model
model = Sequential()

# The number of input features and hidden nodes for each layer
hidden_nodes_1 = 64
hidden_nodes_2 = 32

# First Layer
model.add(Dense(units = hidden_nodes_1, input_dim=X_train_scaled.shape[1], activation='relu'))
#Second Layer
model.add(Dense(units = hidden_nodes_2, activation='relu'))
#Third Layer 
model.add(Dense(units = 1, activation='linear'))

# Check the structure of the model
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                1728      
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 3841 (15.00 KB)
Trainable params: 3841 (15.00 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [14]:
# Compile the model
model.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])

In [15]:
# Train the model
model.fit(X_train_scaled, y_train, epochs=25, batch_size=50, validation_split=0.2, verbose=1)


Epoch 1/25
344/344 [==============================] - 1s 1ms/step - loss: 0.7577 - accuracy: 0.8438 - val_loss: 0.6558 - val_accuracy: 0.8636
Epoch 2/25
344/344 [==============================] - 0s 854us/step - loss: 0.5570 - accuracy: 0.8704 - val_loss: 0.5454 - val_accuracy: 0.8750
Epoch 3/25
344/344 [==============================] - 0s 784us/step - loss: 0.6549 - accuracy: 0.8716 - val_loss: 0.4934 - val_accuracy: 0.8762
Epoch 4/25
344/344 [==============================] - 0s 786us/step - loss: 0.4479 - accuracy: 0.8869 - val_loss: 0.6215 - val_accuracy: 0.8729
Epoch 5/25
344/344 [==============================] - 0s 776us/step - loss: 0.4259 - accuracy: 0.8856 - val_loss: 0.4307 - val_accuracy: 0.8829
Epoch 6/25
344/344 [==============================] - 0s 824us/step - loss: 0.3948 - accuracy: 0.8926 - val_loss: 0.4107 - val_accuracy: 0.8873
Epoch 7/25
344/344 [==============================] - 0s 826us/step - loss: 0.3961 - accuracy: 0.8811 - val_loss: 0.4831 - val_accuracy: 0

In [16]:
# Evaluate the model
model_loss, model_accuracy = model.evaluate(X_test_scaled, y_test)
print("Accuracy = ", model_accuracy)

224/224 [==============================] - 0s 538us/step - loss: 0.3123 - accuracy: 0.8973
Accuracy =  0.8973463773727417


In [17]:
# Convert X_test to float
X_test = X_test.astype(float)

In [18]:
# Make predictions
y_pred = model.predict(X_test)

224/224 [==============================] - 0s 445us/step


In [19]:
# Check accuracy

categorical_predictions = np.argmax(y_pred, axis=1)

accuracy = accuracy_score(y_test, categorical_predictions)

print(accuracy)

0.7777932960893855
